## 2.2 文本标签化(tokenize)

- 载入源文件
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) 一本无版权的短篇小说

- 读取文件内容

In [5]:
with open("data/the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()
print("total number of characters in the verdict:",len(raw_text))
print(raw_text[:99])

total number of characters in the verdict: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 文字分词处理示例

In [6]:
import re

text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;_!?"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 小说分词示例

In [7]:
processed = re.split(r'([,.:;_!?"()\']|--|\s)', raw_text)
processed = [item.strip() for item in processed if item.strip()]
print("total number of words in the verdict:",len(processed))
print(processed[:30])

total number of words in the verdict: 4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## 2.3 给token编号

- 我们要创建一个表格,给所有的token给映射到不同的标号上

In [8]:
all_words = sorted(set(processed))#从去掉重复的字符
vocab_size = len(all_words)#计总的单词书
print(vocab_size)

1130


- 看一下前50个长什么样

In [9]:
all_words = sorted(set(processed))
vocab_size = len(all_words)
print("vocab size:", vocab_size)

vocab = {token:integer for integer, token in enumerate(all_words)}
#查看前50个词
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

vocab size: 1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 现在将所有内容整合到一个分词器中

In [10]:
class SimpleTokenizerV1:#一个实例的名字创立
    def __init__(self, vocab): ## 初始化一个字符串
        self.str_to_int = vocab #单词到整数的映射
        self.int_to_str = {i:s for s,i in vocab.items()}
        #方便解码,进行整数到词汇的反向映射

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)##正则化分词标点符号
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]##整理完的额字符串列表对应到id,从字典出来
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) #映射整数id到字符串。join是用前面那个(“ ”)联结成一个完整的字符串
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #使用正则表达式，去除标点符号前的多余空格
        # \s+匹配一个或者多个空白  \1 替换到匹配
        return text

- `encode` 函数将文本转换为标记 ID。
- `decode` 函数将标记 ID 转换回文本。
- 我们可以使用分词器将文本编码（即分词）为数字。
- 然后，这些整数可以作为大语言模型（LLM）的输入，进行嵌入。

In [11]:
tokenizer = SimpleTokenizerV1(vocab) #用vocab创造一个实例

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text) #按照这个例子里的encode函数处理text
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 我们也可以使用分词器将数字解码（即反向操作）为文本。

In [12]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [13]:
print(tokenizer.decode(tokenizer.encode(text)))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


到目前为止，一切都很顺利。我们实现了一个分词器，能够根据训练集中的片段对文本进行分词和去分词。现在让我们将其应用于训练集中未包含的新文本样本：

In [14]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

问题在于短篇小说《判决》中没有使用“Hello”这个词。因此，它不包含在词汇中。这突显了在处理大型语言模型时，需要考虑大型和多样化的训练集以扩展词汇的必要性。

## 2.4 添加特殊上下文token

修改词汇表，将2个特殊token添加到词汇表中。

In [16]:
all_tokens = sorted(list(set(processed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

print("vocab size:",len(vocab.items()))

vocab size: 1132


为了快速检查，让我们打印更新后词汇表的最后5个条目

In [17]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


根据上述代码的输出，我们可以确认这两个新的特殊token确实成功地被纳入了词汇表。接下来，我们相应地调整代码清单2.3中的分词器，如清单2.4所示：

In [21]:
class SimpleTokenizerV2:  #一个实例的名字创立
    def __init__(self, vocab):  ## 初始化一个字符串
        self.str_to_int = vocab  #单词到整数的映射
        self.int_to_str = {i: s for s, i in vocab.items()}
        #方便解码,进行整数到词汇的反向映射

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)  ##正则化分词标点符号
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in                            preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]  ##整理完的额字符串列表对应到id,从字典出来
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])  #映射整数id到字符串。join是用前面那个(“ ”)联结成一个完整的字符串
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)  #使用正则表达式，去除标点符号前的多余空格
        # \s+匹配一个或者多个空白  \1 替换到匹配
        return text

与在上一节的代码清单 2.3 中实现的 SimpleTokenizerV1 相比，新的 SimpleTokenizerV2 用 <|unk|> token 替换未知词。

In [19]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of palace."
text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of palace.


- 使用之前在2.2中创建的词汇表，通过SimpleTokenizerV2对示例文本进行分词

In [22]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 1131, 7]


看一下解码结果

In [23]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of <|unk|>.


通过将上面的去token化文本与原始输入文本进行比较，我们可以得知训练数据集，即艾迪丝·华顿的短篇小说《判决》，并不包含单词 "Hello" 和 "palace"

## 2.5 字节对编码（Byte pair encoding）

从零开始实现BPE可能相对复杂，我们将使用一个名为tiktoken的现有Python开源库(https://github.com/openai/tiktoken)，该库基于Rust中的源代码非常高效地实现了BPE算法。与其他Python库类似，我们可以通过Python的pip安装程序从终端安装tiktoken库：

In [1]:
pip install tiktoken

   ---------------------------------------- 0.0/878.7 kB ? eta -:--:--
   ---------------------------------------- 878.7/878.7 kB 7.9 MB/s eta 0:00:00

   ---------------------------------------- 2/2 [tiktoken]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


查看当前版本的tiktoken库

In [2]:
import tiktoken
print("tiktoken version:",tiktoken.__version__)

tiktoken version: 0.12.0


安装完成后，我们可以按如下方式通过tiktoken实例化BPE分词器：



In [4]:
tokenizer = tiktoken.get_encoding("gpt2")

似于我们之前实现的 SimpleTokenizerV2，都是通过 encode 方法使用：

In [6]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


我们可以使用 decode 方法将token ID 列表转换回文本，类似于我们之前实现的 SimpleTokenizerV2 类的 decode 方法：

In [7]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


> [!TIP]
>
> **个人思考：** 字节对编码是一种基于统计的方法，它会先从整个语料库中找出最常见的字节对（byte pair），然后把这些字节对合并成一个新的单元。让我们用一个具体的示例来描述这个过程：
>
> 假如有句子：“The cat drank the milk because it was hungry”
>
> 1. **初始化：BPE会先将句子中每个字符视为一个单独的token**
>
>    ```
>    ['T', 'h', 'e', ' ', 'c', 'a', 't', ' ', 'd', 'r', 'a', 'n', 'k', ' ', 't', 'h', 'e', ' ', 'm', 'i', 'l', 'k', ' ', 'b', 'e', 'c', 'a', 'u', 's', 'e', ' ', 'i', 't', ' ', 'w', 'a', 's', ' ', 'h', 'u', 'n', 'g', 'r', 'y']
>    ```
>
> 2. **统计最常见的字节对**
>
>    BPE算法会在这些token中找到出现频率最高的“字节对”（即相邻的两个字符），然后将其合并为一个新的token。
>
>    例如这里最常见的字节对时（'t', 'h'），因为它在单词"the"和"that"中出现频率较高。
>
> 3. **合并字节对**
>
>    根据统计结果，我们将最常见的字节对（'t', 'h'）合并为一个新的token，其它类似
>
>    ```
>    ['Th', 'e', ' ', 'c', 'a', 't', ' ', 'dr', 'a', 'nk', ' ', 'th', 'e', ' ', 'm', 'i', 'l', 'k', ' ', 'be', 'c', 'a', 'u', 'se', ' ', 'it', ' ', 'wa', 's', ' ', 'hu', 'n', 'gr', 'y']
>    ```
>
> 4. **重复步骤2和3，得到最终的token序列**
>
>    ```
>    ['The', ' ', 'cat', ' ', 'drank', ' ', 'the', ' ', 'milk', ' ', 'because', ' ', 'it', ' ', 'was', ' ', 'hungry']
>    ```

## 2.6 使用滑动窗口进行数据采样